In [ ]:
import numpy as np
import os
import sys
import pickle as pkl
import pandas as pd

In [ ]:
sys.path.append('src')
from datasets import dataset_names

In [ ]:
datasets = dataset_names
print(datasets)

In [ ]:
data_dir = 'data'

In [ ]:
algo_names = ['LR', 'LR-CV', 'TP-MLC']

In [ ]:
metrics = ['Precision@3', 'Precision@5', 'Precision@K', 'F1']

In [ ]:
indices = pd.MultiIndex.from_product([dataset_names, algo_names], names=['Dataset', 'Method'])

In [ ]:
#df_train = pd.DataFrame(index=indices, columns=metrics)
df_test  = pd.DataFrame(index=indices, columns=metrics)

In [ ]:
fprec_tp = os.path.join(data_dir, 'perf-tp.pkl')
fprec_lr0 = os.path.join(data_dir, 'perf-lr-base.pkl')
fprec_lr1 = os.path.join(data_dir, 'perf-lr-prec.pkl')
records_tp = pkl.load(open(fprec_tp, 'rb'))
records_lr0 = pkl.load(open(fprec_lr0, 'rb'))
records_lr1 = pkl.load(open(fprec_lr1, 'rb'))

In [ ]:
#records_lr1

In [ ]:
for metric in metrics:
    #df_train.loc[(ds, 'TP-MLC'), metric] = records[ds]['Train'][metric][0]
    if metric == 'F1':
        for ds in datasets:
            if ds in records_tp:
                value = ', '.join(['%.4f' % v[0] for v in records_tp[ds]['Test' ][metric]])
                df_test.loc[(ds, 'TP-MLC'), metric] = value
            if ds in records_lr0:
                df_test.loc[(ds, 'LR'), metric] = records_lr0[ds]['Test' ][metric][0]
            if ds in records_lr1:
                df_test.loc[(ds, 'LR-CV'), metric] = records_lr1[ds]['Test' ][metric][0]
    else:
        for ds in datasets:
            if ds in records_tp:
                df_test.loc[(ds, 'TP-MLC'), metric] = records_tp[ds]['Test' ][metric][0]
            if ds in records_lr0:
                df_test.loc[(ds, 'LR'), metric] = records_lr0[ds]['Test' ][metric][0]
            if ds in records_lr1:
                df_test.loc[(ds, 'LR-CV'), metric] = records_lr1[ds]['Test' ][metric][0]

In [ ]:
tab_test = df_test.to_latex(float_format=lambda x: '%.4f' % x, na_rep='-', multirow=True)

In [ ]:
print('\\begin{table}[!h]')
print('\centering')
print('\\caption{Performance on test set}')
print('\\label{tab:perf}')    
print(tab_test)
print('\\end{table}')

In [ ]:
metrics_f1 = 

In [ ]:
datasets = ['yeast', 'scene', 'bibtex', 'bookmarks']

In [ ]:
#split = 'Train'
split = 'Test'

In [ ]:
# load perf records
perf_files = [os.path.join(data_dir, 'perf-' + algo + '.pkl') for algo in algos]
perf_dicts = [pkl.load(open(fname, 'rb')) for fname in perf_files]

def perfstr(perftuple):
    return '$%.4f \\pm  %.3f$' % (perftuple[0], perftuple[1])

for metric in metrics:
    tabstr = []
    tabstr.append('\\begin{table}[!h]')
    tabstr.append('\centering')
    tabstr.append('\\caption{Performance in terms of average %s (%s)}' % (metric, split))
    tabstr.append('\\begin{tabular}{lcccc} \\\\ \\hline \\hline')
    tabstr.append(' & ' + ' & '.join([ds for ds in datasets]) + ' \\\\')
    for j in range(len(algos)):
        tabstr.append(algo_names[j] + ' & '
                      + ' & '.join([perfstr(perf_dicts[j][ds][split][metric]) \
                                    if ds in perf_dicts[j] else '-' for ds in datasets ])
                      + ' \\\\')
    tabstr.append('\\hline')
    tabstr.append('\\end{tabular}')
    tabstr.append('\\end{table}')
    print('\n'.join(tabstr))
    print()